In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
%pip install pytorch-lightning -q -U

In [17]:
mapping = {
  'B-ORG':'B-Group', 'B-TechCorp':'B-Group', 'B-CarManufacturer':'B-Group', 'B-SportsGRP':'B-Group', 'B-AerospaceManufacturer':'B-Group', 'B-OtherCorp':'B-Group', 'B-PrivateCorp':'B-Group', 'B-PublicCorp':'B-Group', 'B-MusicalGRP':'B-Group',
  'B-VisualWork':'B-CreativeWorks', 'B-MusicalWork':'B-CreativeWorks', 'B-WrittenWork':'B-CreativeWorks', 'B-ArtWork':'B-CreativeWorks', 'B-Software':'B-CreativeWorks', 'B-OtherCW':'B-CreativeWorks',
  'B-OtherPER':'B-Person', 'B-SportsManager':'B-Person', 'B-Cleric':'B-Person', 'B-Politician':'B-Person', 'B-Athlete':'B-Person', 'B-Artist':'B-Person', 'B-Scientist':'B-Person',
  'B-Disease':'B-Medical', 'B-Symptom':'B-Medical', 'B-AnatomicalStructure':'B-Medical', 'B-MedicalProcedure':'B-Medical', 'B-Medication/Vaccine':'B-Medical', 
  'B-OtherPROD':'B-Product', 'B-Drink':'B-Product', 'B-Food':'B-Product', 'B-Vehicle':'B-Product', 'B-Clothing':'B-Product',
  'B-Facility':'B-Location', 'B-OtherLOC':'B-Location', 'B-HumanSettlement':'B-Location', 'B-Station':'B-Location',

  'I-Facility':'I-Location', 'I-OtherLOC':'I-Location', 'I-HumanSettlement':'I-Location', 'I-Station':'I-Location',
  'I-OtherPROD':'I-Product', 'I-Drink':'I-Product', 'I-Food':'I-Product', 'I-Vehicle':'I-Product', 'I-Clothing':'I-Product',
  'I-Disease':'I-Medical', 'I-Symptom':'I-Medical', 'I-AnatomicalStructure':'I-Medical', 'I-MedicalProcedure':'I-Medical', 'I-Medication/Vaccine':'I-Medical', 
  'I-OtherPER':'I-Person', 'I-SportsManager':'I-Person', 'I-Cleric':'I-Person', 'I-Politician':'I-Person', 'I-Athlete':'I-Person', 'I-Artist':'I-Person', 'I-Scientist':'I-Person',
  'I-VisualWork':'I-CreativeWorks', 'I-MusicalWork':'I-CreativeWorks', 'I-WrittenWork':'I-CreativeWorks', 'I-ArtWork':'I-CreativeWorks', 'I-Software':'I-CreativeWorks', 'I-OtherCW':'I-CreativeWorks',
  'I-ORG':'I-Group', 'I-TechCorp':'I-Group', 'I-CarManufacturer':'I-Group', 'I-SportsGRP':'I-Group', 'I-AerospaceManufacturer':'I-Group', 'I-OtherCorp':'I-Group', 'I-PrivateCorp':'I-Group', 'I-PublicCorp':'I-Group', 'I-MusicalGRP':'I-Group',

  '_ O':'_ O', 'O':'O',
  '':''
}

In [18]:
# Preprocess the Penn Treebank dataset
# Load the dataset
import torch

def read_data(file_path):
    parsed_data = []
    with open(file_path, 'r', encoding='utf-8') as f:
      parsed_tokens = []
      for line in f:
            if line == '\n' or line[0] == '#':
                if parsed_tokens:
                  parsed_data.append(parsed_tokens)
                parsed_tokens = []
            else:
                parts = line.strip().split(' _ _ ')
                word = parts[0]
                tag = parts[1] if len(parts) > 1 else ''
                tag = mapping[tag]
                parsed_tokens.append((word, tag))
    return parsed_data

train_dataset= read_data('/content/drive/MyDrive/DeepLearning/multiconer2023/EN-English/en_train.conll')
dev_dataset= read_data('/content/drive/MyDrive/DeepLearning/multiconer2023/EN-English/en_dev.conll')
test_dataset= read_data('/content/drive/MyDrive/DeepLearning/multiconer2023/EN-English/en_test.conll')

for i in range(100):
    print(train_dataset[i])
 

[('robert', 'B-Person'), ('gottschalk', 'I-Person'), ('1939', 'O'), ('academy', 'B-CreativeWorks'), ('award', 'I-CreativeWorks'), ('winner', 'O'), ('and', 'O'), ('founder', 'O'), ('of', 'O'), ('panavision', 'B-Group')]
[('during', 'O'), ('the', 'O'), ('reign', 'O'), ('of', 'O'), ('the', 'O'), ('tongzhi', 'B-Person'), ('emperor', 'I-Person'), ('(', 'O'), ('r', 'O'), ('.', 'O'), ('1861', 'O'), ('–', 'O'), ('1875', 'O'), (')', 'O'), (':', 'O')]
[('further', 'O'), ('research', 'O'), ('led', 'O'), ('in', 'O'), ('the', 'O'), ('1960s', 'O'), ('to', 'O'), ('the', 'O'), ('bahadur', 'B-Person'), ('representation', 'O'), ('which', 'O'), ('provides', 'O'), ('information', 'O'), ('about', 'O'), ('the', 'O'), ('errorbounds', 'O'), ('.', 'O')]
[('the', 'O'), ('ideas', 'O'), ('were', 'O'), ('introduced', 'O'), ('by', 'O'), ('william', 'B-Person'), ('burnside', 'I-Person'), ('at', 'O'), ('the', 'O'), ('end', 'O'), ('of', 'O'), ('the', 'O'), ('nineteenth', 'O'), ('century', 'O'), ('.', 'O')]
[('thaddeus

In [19]:
SEQ_LEN = 30

# Create word_to_idx and tag_to_idx mappings
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
tag_to_idx = {"<PAD>": 0}


def preprocess(dataset):
    # Extract sentences and tags
    sent = [[token.lower() for token, tag in sentence] for sentence in dataset]
    tags = [[tag for token, tag in sentence] for sentence in dataset]

    for i in range(len(sent)):
        while len(sent[i]) < SEQ_LEN:
            sent[i].append('<PAD>')
            tags[i].append('<PAD>')

        if len(sent[i]) > SEQ_LEN:
            sent[i] = sent[i][:SEQ_LEN]
            tags[i] = tags[i][:SEQ_LEN]
    
    for sentence_tags in tags:
        for tag in sentence_tags:
            if tag not in tag_to_idx:
                tag_to_idx[tag] = len(tag_to_idx)
    
    for sentence in sent:
        for word in sentence:
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)

    # Convert words and tags to indices
    X = torch.tensor([[word_to_idx.get(word, 1) for word in sentence] for sentence in sent], dtype=torch.int).type(torch.LongTensor)
    Y = torch.tensor([[tag_to_idx[tag] for tag in sentence] for sentence in tags], dtype=torch.int).type(torch.LongTensor)
    
    return X, Y

In [20]:
train_X, train_Y = preprocess(train_dataset)
dev_X, dev_Y = preprocess(dev_dataset)
test_X, test_Y = preprocess(test_dataset)

In [21]:
# Print the sizes of the datasets
print(f"Number of training examples: {len(train_X)}")
print(f"Number of validation examples: {len(dev_X)}")
print(f"Number of testing examples: {len(test_X)}")
# for i in range(100):
#     print(train_X[i])

Number of training examples: 16778
Number of validation examples: 871
Number of testing examples: 249980


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class NERModel(pl.LightningModule):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim) #B * seq_len, B * seq_len * embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=num_layers, bidirectional=True)
        self.fc = nn.Linear(2*hidden_dim, tagset_size)
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, x):
        embeds = self.embedding(x)
        #print(embeds.shape)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.fc(lstm_out)
        tag_scores = nn.functional.log_softmax(tag_space, dim=2)
        return tag_scores
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer

In [23]:
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
EMBEDDING_DIM = 200
HIDDEN_DIM    = 200
NUM_EPOCHS    = 10 
BATCH_SIZE    = 10

train_dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(dev_X, dev_Y)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [24]:
model = NERModel(vocab_size=len(word_to_idx), tagset_size=len(tag_to_idx), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, callbacks=[early_stopping])
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

trainer.test(dataloaders=test_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 48.4 M
1 | lstm      | LSTM             | 643 K 
2 | fc        | Linear           | 5.6 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
49.1 M    Trainable params
0         Non-trainable params
49.1 M    Total params
196.318   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_1/checkpoints/epoch=4-step=8390.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_1/checkpoints/epoch=4-step=8390.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.30375123023986816    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.30375123023986816}]

In [25]:
from sklearn.metrics import classification_report

# define idx_to_tag
idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

# define device
device = torch.device('cpu')

# Create a dataloader for the test set
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Set the model to evaluation mode
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

print(classification_report(y_true, y_pred))

                 precision    recall  f1-score   support

          <PAD>       1.00      1.00      1.00   3727316
B-CreativeWorks       0.54      0.39      0.45     62124
        B-Group       0.53      0.44      0.48     60026
     B-Location       0.71      0.54      0.61     67893
      B-Medical       0.45      0.25      0.32     22490
       B-Person       0.75      0.75      0.75    137666
      B-Product       0.34      0.19      0.24     27574
I-CreativeWorks       0.69      0.47      0.56    107463
        I-Group       0.64      0.50      0.56     74136
     I-Location       0.78      0.61      0.68     63007
      I-Medical       0.58      0.27      0.37     10613
       I-Person       0.81      0.74      0.77    153751
      I-Product       0.36      0.17      0.23     17503
              O       0.92      0.97      0.95   2967838

       accuracy                           0.94   7499400
      macro avg       0.65      0.52      0.57   7499400
   weighted avg       0.93   

In [26]:
# Set the model to evaluation mode
model.eval()

idx_to_word = {idx: word for word, idx in word_to_idx.items()}

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)

        # Get back the sentence
        x_sent = [idx_to_word[i] for i in x.cpu().numpy().flatten().tolist()]

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]
        print("Sentence\t:", end = ' ')
        for i in range(len(x_sent)):
          print(x_sent[i], end = ' ')
        print("\nPredicted tags\t:", end = ' ')
        for i in range(len(y_pred)):
          print(y_pred[i], end = ' ')
        break

Sentence	: the species was described by dietrich brandis after the forester t. f. bourdillon . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> by this time she was competing against a new generation of young drivers including stirling moss and peter collins . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> their son was the opera producer knut hendriksen ( 1944 – 2020 ) . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> he is the younger brother of adam mosseri . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> smes : laura j. van 't veer et al . ( nl ) for their gene based breast cancer test <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> pau became a leading political and intellectual centre under the reign of henry d'albret . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

In [27]:
torch.save(model.state_dict(), '/content/drive/MyDrive/DeepLearning/multiconer2023/EN-English/English_coarse.pth')

In [28]:
eng_model = model.load_state_dict(torch.load('/content/drive/MyDrive/DeepLearning/multiconer2023/EN-English/English_coarse.pth'))
def get_prediction(sentence):
    # Split the sentence into words
    words = sentence.split()
    for word in words:
      word = word.lower()

    # Get the index of the words
    X = []
    for word in words:
        if word not in word_to_idx:
            X.append(word_to_idx["<UNK>"])
        else:
            X.append(word_to_idx[word])
    
    # Create a tensor
    X = torch.tensor(X).unsqueeze(0)
    
    # Get the predictions
    y_hat = model(X)
    
    # Get the predictions
    predictions = [idx_to_tag[pred] for pred in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

    # Give new sentence
    new_sentence = []
    flag = False
    tag = ""
    for word, pred in zip(words, predictions):
        if pred == "O":
            if flag:
                new_sentence[new_sentence.__len__() - 1] = new_sentence[new_sentence.__len__() - 1] + "[" + tag + "]"
                flag = False
                tag = ""
            new_sentence.append(word)
        elif len(pred) == 0:
            new_sentence.append(word)
        elif pred[0] == "B":
            flag = True
            tag = pred[2:]
            new_sentence.append(word)
        else:
            new_sentence.append(word)
    if flag:
        new_sentence[new_sentence.__len__() - 1] = new_sentence[new_sentence.__len__() - 1] + "[" + tag + "]"
        flag = False
        tag = "" 
    
    return " ".join(new_sentence)
print(get_prediction("Jim bought 300 shares of Acme Corp. in 2022."))


Jim bought 300 shares of Acme Corp.[Location] in 2022.[Location]
